## Brief Problem Description
Given an input file which has the location of a person based on cellular tower triangulation, generate an output with 2 columns,
containing one row for each input line, tagging it as a tower jump (erroneous localization) and a confidence level (expressed as a number between 0 and 1)

## References

https://www.tinybird.co/blog-posts/anomaly-detection

https://medium.com/analytics-vidhya/removing-outliers-understanding-how-and-what-behind-the-magic-18a78ab480ff


## Solution (TLDR)

1. if lat and lon are zero -> flag as jump ( I hope John Doe is not visiting [Null Island](https://en.wikipedia.org/wiki/Null_Island) )
2. order the full dataset by datetime
3. calculate diff in minutes between points
4. calculate quartile of 80% in min (this was selected by me at random, it will be a variable in case we want to play with it)
5. group measurements in cluster when the time in minutes between rows is below the 80% quartile
6. remove outliers, calculate z value for lat and lon, any row that has either lon or lat below z +- 2 std, will be flagged as jump
7. calculate confidence as how close the lat and lon are to z if it is not a jump, or 1 - diff if it was flagged as jump
8. reorder output based on page and item number

## This Notebook
... was used when attempting to extract relevant data from the sample to write the solution, so it is not necessarily in an order
to help understand the solution, but in the order the data was being analysed.

## Initial Assumptions
Given the nature of the problem, which does not appear to have an exact solution that can be solved by a formal algorithm (the request to use a "confidence level" in the output gave that hint), we will try some statistical approach.
1- This appears to be a particular case of anomaly detection, and it will be handled as such
2- Many simplifications will be done given the limitations of time
3- A Script will be provided that can be called on the command line to produce the output, given the input is in the same format that the one provided in the challenge description

### Loading input
First load the input file and check the content:

In [ ]:
import pandas as pd

INPUT_PATH = "TowerJumpsDataSet_CarrierRecords.csv"

df = pd.read_csv(INPUT_PATH)
df.head(5)

Now, we look at all values in the columns to try to get a perspective of how this data is distributed.
By running `describe`, nothing particular draws our attention, except that the page number and item number do not seem relevant to our analysis.

In [ ]:
df.describe()

Time zone appears to have 2 exotic values: Etc and Asia. Let´s look a bit closer at what data they have.

In [ ]:
df["Time Zone"].value_counts()

Asia has only 5 entries, the location (lat and lon) might be correct, though.

In [ ]:
df[df["Time Zone"]=="Asia/Bishkek"].head(10)

ETC has no good value for lat and lon, so this one can possibly be a case for automatically flagging it as a jump with high confidence.

In [ ]:
df[df["Time Zone"]=="Etc/GMT"].head(10)

In [ ]:
df[df["Time Zone"]=="Etc/GMT"][["Latitude","Longitude"]].describe()

Country is either USA or NaN, so nothing extraordinary here.

In [ ]:
df["Country"].value_counts()

States look OK, except for the `unknown` value. However, some of the `unknown` have non zero lat/lon location, so this alone is possibly not a good indicator of a jump.

In [ ]:
df["State"].value_counts()

In [ ]:
df[df["State"]=="unknown"].head(10)

In [ ]:
df[df["State"]=="unknown"][["Latitude","Longitude"]].describe()

Record Type does not seem to give valuable information for our task.

In [ ]:
df["Record Type"].value_counts()

Datetime can be used to gather interesting information, like the distribution of the records over time.

In [ ]:
df["Datetime"] = pd.to_datetime(df["Local Date & Time"], format="%m/%d/%y %H:%M")

In [ ]:
df = df.sort_values(by=["Datetime"])

In [ ]:
df["Datetime"].describe()

In [ ]:
cleansed_df = df[df["Datetime"].notnull()]

In [ ]:
df["DatetimeDelta"] =  cleansed_df["Datetime"] - cleansed_df["Datetime"].shift(1)
df["DatetimeDelta"] = df["DatetimeDelta"].fillna(0)

## DatetimeDeltaMinutes
This is an interesting metric because it shows the distribution of data.
In practise, this means that the effect that many records with the same location happen in a short period of time, in bursts,
is very common in this dataset.
I do not have any expertise in cellular tower location algorithms, but statiscally, we will consider that if a position is reported
repeatedly, this will mean it is more relevant and has a higher chance of being correct.

In [ ]:
df["DatetimeDeltaMinutes"] = df["DatetimeDelta"].apply(lambda x: (x.total_seconds())/60 if x else 0)

In [ ]:
df[["Datetime","DatetimeDelta","DatetimeDeltaMinutes"]].head(10)

In [ ]:
df["DatetimeDeltaMinutes"].describe()

Now we can visualize that the vast majority of interval between measurements is actually pretty small, and also that
the 90% quartile is 16 minutes.

In [ ]:
import seaborn as sns
sns.histplot(data=df[["Datetime","DatetimeDeltaMinutes"]], x="DatetimeDeltaMinutes",binwidth=5)

In [ ]:
df["DatetimeDeltaMinutes"].quantile(0.9)

In [ ]:
quartile_limit = df["DatetimeDeltaMinutes"].quantile(0.8)
quartile_limit

## Intuition and Main code
Reading the references regarding anomaly detection, and taking into account the pecualirity of ths data set regarding the
"burst" effect (there are many measurements very close to each other in time, and those groups are separated by a large amount of time), instead of using a fixed time window to calculate the Z value each point and judge if a point is an anomaly or not,
we will instead cluster the points together when they happen close to each other in time.

The idea is that if a measurement happens in a single geo location and there are no conflicting measurements at the same time, we do not have a lot of reason to believe it is faulty, but if many measurements happen with different locations and close to each other in terms of time, the chance that some of them are faulty is higher.
We have one specific case that is always tagged as faulty which is lat and lon = (0,0).

There were some parameter tuning made while looking at the results to optimize the desired behavior.
1. the value used to classify a point as either faulty or not was 1 standard deviation. This is lower than the suggested value of 2 because we want to have one tagged as correct and many different tagged as faulty when there are conflicts.
2. the upper and lower bounds used to remove outliers were 15 quantile and 85 quantile, because they had better result than 25 and 75.
3. the calculation of accuracy was done manually, following the logic that z values close to 0 should have higher accuracy, whereas values higher than 2 should have very low accuracy. At the same time, if we flag a point as faulty and the z value is really big, accuracy should be higher.
4. the calculation of difference between locations was done using simple euclidean distance instead of using proper geolocation calculation. This was required to calculate the Z Score for each point.

Another important aspect we took into account is that faulty measurements should be the exception, not the rule, otherwise this method of geolocation would make sense. In other words, there should be generally more correct measurements than faulty ones.

With those concepts in mind, the following code was written to calculate the Z Score to each point and tag it as `tower_jump` = y or n.

**Note that the code below will run only on 1000 entries. This was done on purpose to just validate the code. The full version of the code is available in the `main.py` file**

In [ ]:
from typing import Tuple
class LocationPoint:
    def __init__(self, location:Tuple[float,float], page:int, item:int, date_and_time:str):
        self.location = location
        self.page = page
        self.item = item
        self.date_and_time = date_and_time
        self.z_dist = None
        self.tower_jump = None
        self.accuracy = None
        self.cluster_avg = None

    def __str__(self):
        return f"Location: {self.location} \n Date_and_time: {self.date_and_time} \n Page: {self.page} \n Item: {self.item} \n Z_dist: {self.z_dist} \n Tower_jump: {self.tower_jump} \n Accuracy: {self.accuracy} \n Cluster AVG: {self.cluster_avg})"

    def __repr__(self):
        return self.__str__()

    def to_dict(self):
        return {
            "page":self.page,
            "item":self.item,
            "date_and_time":self.date_and_time,
            "tower_jump": self.tower_jump,
            "accuracy": "{:.2f}".format(float(self.accuracy))
        }

    @staticmethod
    def from_row(input):
        lat = input["Latitude"]
        lon = input["Longitude"]
        if isinstance(lat, np.float64):
            lat = float(lat)
        if isinstance(lon, np.float64):
            lon = float(lon)
        return LocationPoint((lat,lon),input["Page Number"],input["Item Number"],input["Local Date & Time"])
        

In [ ]:
def calculate_accuracy_from_z(z_score:float)->int:
    if abs(z_score) <= 0.5:
        return 80 + (0.5 - abs(z_score))*38
    elif abs(z_score) <= 1:
        return 50 + (1 - abs(z_score))*30
    elif abs(z_score) <= 2:
        return 30 + (2 - abs(z_score))*10
    else:
        return 0

In [ ]:
from typing import List
import numpy as np

def calculate_mean_std_and_z_scores(input_list:List[LocationPoint]) -> List[LocationPoint]:

    all_zeros = all([item.location == (0,0) for item in input_list])

    if not all_zeros:
        lat_list = []
        lon_list = []
        for item in input_list:
            if item.location != (0,0):
                lat_list.append(item.location[0])
                lon_list.append(item.location[1])
    
        #print(f"LAT {lat_list} LON {lon_list}")
        # remove outliers
        q1_lat = np.percentile(lat_list, 15)
        q3_lat = np.percentile(lat_list, 85)
        q1_lon = np.percentile(lon_list, 15)
        q3_lon = np.percentile(lon_list, 85)
    
        # print(f"PERCENTILE LAT {q1_lat} {q3_lat} LON {q1_lon} {q3_lon}")
    
        clean_lat_list = [item for item in lat_list if item >= q1_lat and item <= q3_lat]
        clean_lon_list = [item for item in lon_list if item >= q1_lon and item <= q3_lon]
        #print(f"CLEAN LAT {clean_lat_list} LON {clean_lon_list}")
    
        lat_avg = np.average(clean_lat_list)
        lon_avg = np.average(clean_lon_list)
    
        std_lat = np.std(clean_lat_list)
        std_lon = np.std(clean_lon_list)
        #print(f"STD {std_lat} {std_lon}")
    
        total_std = np.sqrt(np.sum(np.power([std_lat,std_lon],2)))
    
        #print(f"TOTAL STD {total_std}")

    for item in input_list:
        if item.location == (0,0):
            item.z_dist = -10
            item.tower_jump = "y"
            item.accuracy = 99
        else:
            item.z_dist = np.linalg.norm(np.subtract(list(item.location),[lat_avg,lon_avg])) / total_std if total_std > 0 else 0
            #print(f"Z_DIST {item.z_dist}")
            item.tower_jump = "y" if abs(item.z_dist) > 1 else "n"
            item.accuracy = calculate_accuracy_from_z(item.z_dist) if item.tower_jump == "n" else (100 - calculate_accuracy_from_z(item.z_dist))
        item.cluster_avg = (lat_avg,lon_avg) if not all_zeros else None
    return input_list

In [ ]:
from itertools import islice
point_dict_list = []
current_cluster = [LocationPoint.from_row(df.iloc[0])]
for index, row in islice(df.iterrows(), 1, 1000):
    if row["DatetimeDeltaMinutes"] < 5:
        current_cluster.append(LocationPoint.from_row(row))
    else:
        # print(f" CLUSTER TO ANALYSE {current_cluster}")
        point_list = calculate_mean_std_and_z_scores(current_cluster)
        point_dict_list += [item.to_dict() for item in point_list]
        current_cluster = [LocationPoint.from_row(row)]
#print(f"FINAL LIST {point_dict_list}")
        

In [ ]:
result_df = pd.DataFrame(point_dict_list)
result_df.head(5)

In [ ]:
result_df = result_df.sort_values(by=["page","item"],ascending=[True, True])
result_df.head(5)

In [ ]:
result_df.to_csv("output.csv")